<a href="https://colab.research.google.com/github/OneNameSam/mlzoomcamp_work/blob/main/Week_3_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [44]:
features = ['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']


In [45]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv'

In [46]:
df = pd.read_csv(url)

In [47]:
df = df[features].fillna(0)
df.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [48]:
df['above_average'] = df.price >= 152 


In [49]:
df.columns

Index(['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'above_average'],
      dtype='object')

#### Question 1 

In [50]:
df.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
# Splitting the df into train and test 
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
# Splitting the full train df into train and validation dfs
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [53]:
y_train = df_train.price.values 
y_val = df_val.price.values 
y_test = df_test.price.values 

In [54]:
del df_train['price']
del df_val['price']
del df_test['price']

#### Question 2 

In [55]:
#df_full_train[numerical].corrwith(df_full_train.churn).abs()

In [56]:
corrMatrix = df.corr().abs()
display (corrMatrix)

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
latitude,1.000000,0.084788,0.033939,0.024869,0.015389,0.018758,0.019517,0.010983,0.056385
longitude,0.084788,1.000000,0.150019,0.062747,0.059094,0.138516,0.114713,0.082731,0.267001
price,0.033939,0.150019,1.000000,0.042799,0.047954,0.050564,0.057472,0.081829,0.418150
minimum_nights,0.024869,0.062747,0.042799,1.000000,0.080116,0.124905,0.127960,0.144303,0.034715
number_of_reviews,0.015389,0.059094,0.047954,0.080116,1.000000,0.589407,0.072376,0.172028,0.051785
reviews_per_month,0.018758,0.138516,0.050564,0.124905,0.589407,1.000000,0.047312,0.163732,0.056040
calculated_host_listings_count,0.019517,0.114713,0.057472,0.127960,0.072376,0.047312,1.000000,0.225701,0.170075
availability_365,0.010983,0.082731,0.081829,0.144303,0.172028,0.163732,0.225701,1.000000,0.104255
above_average,0.056385,0.267001,0.418150,0.034715,0.051785,0.056040,0.170075,0.104255,1.000000


In [57]:
#df['above_average'] = df.price >= 152 

In [58]:
df

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365,False
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355,True
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,0.00,1,365,False
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194,False
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Private room,40.67853,-73.94995,70,2,0,0.00,2,9,False
48891,Brooklyn,Private room,40.70184,-73.93317,40,4,0,0.00,2,36,False
48892,Manhattan,Entire home/apt,40.81475,-73.94867,115,10,0,0.00,1,27,False
48893,Manhattan,Shared room,40.75751,-73.99112,55,1,0,0.00,6,2,False


### Question 3

In [59]:
from sklearn.metrics import mutual_info_score

In [60]:
df_train.dtypes
categorical = ['neighbourhood_group', 'room_type']
numerical = ['latitude',
'longitude',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']

In [61]:
def mutal_info_churn_score(series):
  return round(mutual_info_score(series, df_train.above_average),2)

In [62]:
mi = df_train[categorical].apply(mutal_info_churn_score)
mi.sort_values(ascending=False)

room_type              0.14
neighbourhood_group    0.05
dtype: float64

### Question 4

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score

In [64]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [65]:
train_dicts[0]

{'availability_365': 50,
 'calculated_host_listings_count': 13,
 'latitude': 40.7276,
 'longitude': -73.94495,
 'minimum_nights': 3,
 'neighbourhood_group': 'Brooklyn',
 'number_of_reviews': 29,
 'reviews_per_month': 0.7,
 'room_type': 'Entire home/apt'}

In [66]:
dv = DictVectorizer(sparse=False)

In [67]:
dv.fit(train_dicts)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
               sparse=False)

In [68]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']

In [69]:
X_train = dv.fit_transform(train_dicts)
X_train.shape

(29337, 15)

In [70]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred =model.predict(X_val)

In [72]:
#cross_val_score(y_pred, y_val)
from sklearn import metrics
main_accuracy = round((metrics.r2_score(y_pred, y_val)),2)
main_accuracy

-39.17

### Question 5 

In [73]:
categorical + numerical


['neighbourhood_group',
 'room_type',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [74]:
['neighbourhood_group',
'room_type',
'number_of_reviews',
'reviews_per_month']

['neighbourhood_group', 'room_type', 'number_of_reviews', 'reviews_per_month']

In [75]:
focused_features_list = ['room_type',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

train_dicts = df_train[focused_features_list].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred =model.predict(X_val)
accuracy = round((metrics.r2_score(y_pred, y_val)),2)
accuracy - main_accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


3.8599999999999994

In [76]:
focused_features_list = ['neighbourhood_group',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

train_dicts = df_train[focused_features_list].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred =model.predict(X_val)
accuracy = round((metrics.r2_score(y_pred, y_val)),2)
accuracy - main_accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.75

In [77]:
focused_features_list = ['neighbourhood_group',
                         'room_type',
                         'latitude',
 'longitude',
 'minimum_nights',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

train_dicts = df_train[focused_features_list].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred =model.predict(X_val)
accuracy = round((metrics.r2_score(y_pred, y_val)),2)
accuracy - main_accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.0799999999999983

In [78]:
focused_features_list = ['neighbourhood_group',
                         'room_type',
                         'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'calculated_host_listings_count',
 'availability_365']

train_dicts = df_train[focused_features_list].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred =model.predict(X_val)
accuracy = round((metrics.r2_score(y_pred, y_val)),2)
accuracy - main_accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


1.1799999999999997